In [1]:
import numpy as np
import os
import scipy.io as sio
import gc

In [4]:
def all_file(user_dir):  # 获取指定路径下所有文件的全路径
    file_list = list()
    for root, dirs, files in os.walk(user_dir):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list


matFile = all_file(r"./eeg_feature_smooth")

matFile.sort()
data = []
for i in matFile:
    data.append(sio.loadmat(i))  # 读取该目录下的所有.mat文件
print(matFile)

['./eeg_feature_smooth/10_20151014.mat', './eeg_feature_smooth/10_20151021.mat', './eeg_feature_smooth/10_20151023.mat', './eeg_feature_smooth/11_20150916.mat', './eeg_feature_smooth/11_20150921.mat', './eeg_feature_smooth/11_20151011.mat', './eeg_feature_smooth/12_20150725.mat', './eeg_feature_smooth/12_20150804.mat', './eeg_feature_smooth/12_20150807.mat', './eeg_feature_smooth/13_20151115.mat', './eeg_feature_smooth/13_20151125.mat', './eeg_feature_smooth/13_20161130.mat', './eeg_feature_smooth/14_20151205.mat', './eeg_feature_smooth/14_20151208.mat', './eeg_feature_smooth/14_20151215.mat', './eeg_feature_smooth/15_20150508.mat', './eeg_feature_smooth/15_20150514.mat', './eeg_feature_smooth/15_20150527.mat', './eeg_feature_smooth/1_20160518.mat', './eeg_feature_smooth/1_20161125.mat', './eeg_feature_smooth/1_20161126.mat', './eeg_feature_smooth/2_20150915.mat', './eeg_feature_smooth/2_20150920.mat', './eeg_feature_smooth/2_20151012.mat', './eeg_feature_smooth/3_20150919.mat', './eeg

In [5]:
count=0
SepMatFile = []
temp=[]
for i in matFile:
    temp.append(i)
    if(count % 3 == 2):
        SepMatFile.append(temp)
        temp = []
    count+=1

In [6]:
data = []
for i in SepMatFile:
    singleData = []
    for j in i:
        singleData.append(sio.loadmat(j))  # 读取该目录下的所有.mat文件
    data.append(singleData)

In [14]:
de_keys = []

for i in range(24):
    de_keys.append('de_LDS' + str(i+1))

In [15]:
print(de_keys)

['de_LDS1', 'de_LDS2', 'de_LDS3', 'de_LDS4', 'de_LDS5', 'de_LDS6', 'de_LDS7', 'de_LDS8', 'de_LDS9', 'de_LDS10', 'de_LDS11', 'de_LDS12', 'de_LDS13', 'de_LDS14', 'de_LDS15', 'de_LDS16', 'de_LDS17', 'de_LDS18', 'de_LDS19', 'de_LDS20', 'de_LDS21', 'de_LDS22', 'de_LDS23', 'de_LDS24']


In [16]:
all_length = []
all_de_exp = []

for i in range(len(data)): # Subject循环
    length = []
    de_exp = []
    for j in range(3): # .mat循环
        temp_length = []
        temp_de_exp = []    

        for k in de_keys: # 实验数循环
            temp_de_exp.append(data[i][j][k])
            temp_length.append(data[i][j][k].shape[1])

        length.append(temp_length)
        de_exp.append(temp_de_exp)
    all_length.append(length)
    all_de_exp.append(de_exp)

In [17]:
all_de_exp_arr = np.zeros([15, 3, 24, 62, max(temp_length),5])

In [19]:
len(all_de_exp[0][0])

24

In [20]:
for i in range(15):
    for j in range(3):
        for k in range(24):
            tem = all_de_exp[i][j][k]
            all_de_exp_arr[i][j][k] = np.concatenate(
                (tem, np.zeros([62, max(temp_length) - all_length[i][j][k], 5])), axis=1)

In [21]:
all_de_exp_arr.shape

(15, 3, 24, 62, 64, 5)

In [22]:
all_de_exp_arr = all_de_exp_arr.swapaxes(-2, -3)
rawShape = all_de_exp_arr.shape
print(rawShape)

(15, 3, 24, 64, 62, 5)


In [23]:
de = all_de_exp_arr

In [25]:
de_sub_list = []
for i in range(15):
    de_mat_list = []
    for j in range(3):
        de_trial_list = []
        for k in range(24):
            zero_index = []
            for m in range(64):
                if(de[i][j][k][m].sum() == 0):
                    zero_index.append(m)
            de_trial_list.append(np.delete(de[i][j][k], zero_index, axis=0))
        de_mat_list.append(de_trial_list)
    de_sub_list.append(de_mat_list)

In [27]:
for i in range(15):
    os.mkdir("trials/feature/subject_" + str(i))
    for j in range(3):
        os.mkdir("trials/feature/subject_" + str(i) + "/section_" + str(j))

In [28]:
de_sub_list[0][0][0].shape

(42, 62, 5)

In [29]:
# 电极对应到二维矩阵中，可对应map.xlsx中的图看看
for i in range(15):
    for j in range(3):
        for k in range(24):
            nSample = de_sub_list[i][j][k].shape[0]
            de_headMap = np.zeros([nSample, 9, 9, 5])
            for x in range(nSample):  # 样本数循环
                # 0-1行
                de_headMap[x][0][3] = de_sub_list[i][j][k][x][0]
                de_headMap[x][0][4] = de_sub_list[i][j][k][x][1]
                de_headMap[x][0][5] = de_sub_list[i][j][k][x][2]
                de_headMap[x][1][3] = de_sub_list[i][j][k][x][3]
                de_headMap[x][1][5] = de_sub_list[i][j][k][x][4]

                # 2-5行
                for m in range(2, 7):
                    for n in range(9):
                        de_headMap[x][m][n] = de_sub_list[i][j][k][x][(m - 2) * 9 + n + 5]

                # 第7行
                for m in range(1, 8):
                    de_headMap[x][7][m] = de_sub_list[i][j][k][x][m + 49]

                # 第8行
                for m in range(2, 7):
                    de_headMap[x][8][m] = de_sub_list[i][j][k][x][m + 55]
            np.save("trials/feature//subject_" + str(i) + "/section_" + str(j) + "/trial_temp" + str(k), de_headMap)

In [30]:
de_headMap.shape

(39, 9, 9, 5)

In [31]:
# 设置新的边长
sideLen = 32

In [32]:
from scipy import interpolate

for i in range(15):
    for j in range(3):
        for k in range(24):
            de_headMap = np.load("trials/feature/subject_" + str(i) + "/section_" + str(j) + "/trial_temp" + str(k) + ".npy")
            de_timeFirst = de_headMap.swapaxes(-1, -2).swapaxes(-2, -3)
            deMap_bigger = np.zeros([de_timeFirst.shape[0], 5, sideLen, sideLen])
            for m in range(de_timeFirst.shape[0]):  # 样本数循环
                for n in range(5):  # 频率循环
                    x = np.arange(9)
                    y = np.arange(9)
                    z = de_timeFirst[m][n]  # 取出每一个平面

                    f = interpolate.interp2d(x, y, z, kind='cubic')  # cubic为三次
                    xnew = np.linspace(0, 8, sideLen)
                    ynew = np.linspace(0, 8, sideLen)
                    deMap_bigger[m][n] = f(xnew, ynew)
            np.save("trials/feature/subject_" + str(i) + "/section_" + str(j) + "/trial_interpolated_" + str(k) + ".npy", deMap_bigger)

In [2]:
# 标准化
#sample * time * 32 * 32
for i in range(15):
    for j in range(3):
        for k in range(24):
            eegMap = np.load("trials/feature/subject_" + str(i) + "/section_" + str(j) + "/trial_interpolated_" + str(k) + ".npy")
            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] - eegMap[m][n].mean()

            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] / eegMap[m][n].std()

            # 换轴
            eegMap = eegMap.swapaxes(1, 2).swapaxes(2, 3)

            #添加维度
            raw_shape = list(eegMap.shape)
            raw_shape.append(1)

            eegMap = eegMap.reshape(raw_shape)

            # 降低精度为单精度（float32）
            dt = np.dtype(np.float32)
            eegMap = np.array(eegMap, dtype=dt)
            np.save("final_feature/subject_" + str(i) + "_section_" + str(j) + "_trial_" + str(k) + ".npy", eegMap)